# KoGPT2로 문장 생성하기

런타임 유형을 **GPU**로 설정하세요.

원본 링크 : https://github.com/ukairia777/tensorflow-nlp-tutorial (<a href=https://creativecommons.org/licenses/by-nc-sa/2.0/kr/>CC BY-NC-SA 2.0 KR</a>)<br>
참고 자료 : https://huggingface.co/blog/how-to-generate<br>
Modified by uramoon@kw.ac.kr

KoGPT2는 한국어 데이터로 훈련한 GPT-2 모델입니다. (ChatGPT는 GPT-3.5)<br>
GPT는 Generative Pre-trained Transformer의 약어로 OpenAI에서 개발한 자연어처리 모델입니다.


![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQc4oeBEEOISwWxlXG2X7XViY-2GAmHea9kUQ&usqp=CAU)

본 노트북에서는 SKT에서 만든 KoGPT2를 불러와서 주어진 문자열 뒤에 올 수 있는 자연스러운 문장을 생성합니다.<br> KoGPT2는 모두의 말뭉치, 한국어 위키백과, 뉴스기사, 청와대 국민청원 등 다양한 데이터로 훈련했습니다.

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.8 MB/s eta 0:00:00


## KoGPT2 불러오기

In [2]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2') # 토크나이저
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True) # 모델

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.4.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'lm_head.weight', 'transformer.h.1.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.2.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

## 입력 만들기

In [3]:
sent = '근육이 커지기 위해서는'

In [18]:
input_ids = tokenizer.encode(sent) # 토큰화: 텍스트를 정수로 변환
print(type(input_ids), len(input_ids))
print(input_ids, '\n')

input_ids = tf.convert_to_tensor([input_ids]) # 인공신경망에 들어갈 수 있도록 변환
print(type(input_ids), len(input_ids))
print(input_ids)

<class 'list'> 4
[33245, 10114, 12748, 11357] 

<class 'tensorflow.python.framework.ops.EagerTensor'> 1
tf.Tensor([[33245 10114 12748 11357]], shape=(1, 4), dtype=int32)


## 생성하기

### **Greedy Search**

Greedy search는 매 스텝 가장 확률이 높은 단어를 선택합니다. 

![Greedy Search](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/greedy_search.png)

"$\text{The}$" 다음에 "$\text{nice}$"가 나올 확률이 50%라 "$\text{nice}$"를 선택하고, 그 다음에는 "$\text{woman}$"이 나올 확률이 40%로 제일 높아 "$\text{woman}$"을 선택합니다.

$\text{The, nice, woman}$이 순서대로 나올 확률은 $0.5 \times 0.4 = 0.2$입니다.

In [10]:
# generate 함수를 사용하면 기본적으로 greedy search를 수행합니다.

output = model.generate(input_ids, # 주어진 텍스트
                        max_length=128, # 생성할 문장의 최대 길이
                        repetition_penalty=2.0 # 동일한 문구를 반복하여 생성할 때 부여할 벌점
)

output_ids = output.numpy().tolist()[0]

# 생성한 토큰(정수)들을 출력
print(output_ids)

[33245, 10114, 12748, 11357, 23879, 39306, 9684, 7884, 10211, 15177, 26421, 387, 17339, 7889, 9908, 15768, 6903, 15386, 8146, 12923, 9228, 18651, 42600, 9564, 17764, 9033, 9199, 14441, 7335, 8704, 12557, 32030, 9510, 18595, 9025, 10571, 25741, 10599, 13229, 9508, 7965, 8425, 33102, 9122, 21240, 9801, 32106, 13579, 12442, 13235, 19430, 8022, 12972, 9566, 11178, 9554, 24873, 7198, 9391, 12486, 8711, 9346, 7071, 36736, 9693, 12006, 9038, 10279, 36122, 9960, 8405, 10826, 18988, 25998, 9292, 7671, 9465, 7489, 9277, 10137, 9677, 9248, 9912, 12834, 11488, 13417, 7407, 8428, 8137, 9430, 14222, 11356, 10061, 9885, 19265, 9377, 20305, 7991, 9178, 9648, 9133, 10021, 10138, 30315, 21833, 9362, 9301, 9685, 11584, 9447, 42129, 10124, 7532, 17932, 47123, 37544, 9355, 15632, 9124, 10536, 13530, 12204, 9184, 36152, 9673, 9788, 9029, 11764]


In [15]:
print(type(output))

len(output.numpy().tolist()[0])

<class 'tensorflow.python.framework.ops.EagerTensor'>


128

In [6]:
# 정수를 사람이 알아볼 수 있는 형태로 decode
tokenizer.decode(output_ids)

'근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.\n특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.\n또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다.\n아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다.\n운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다.\n운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다.\n운동을'

### **Beam search**

Beam search는 최종적으로 생성될 확률이 가장 높은 경로를 선택하여 문장을 생성합니다. 주어진 텍스트 다음에 두 개의 단어까지만 고려하는 예제를 살펴보도록 하겠습니다. (`num_beams=2`)

![Beam search](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/beam_search.png)

$\text{The, nice, woman}$이 순서대로 나올 확률은 $0.5 \times 0.4 = 0.2$이지만, $\text{The, dog, has}$가 순서대로 나올 확률은 $0.4 \times 0.9 = 0.36$이기 때문에 Beam search는 $\text{The, nice, woman}$ 대신 $\text{The, dog, has}$를 생성합니다.

In [7]:
# TODO: 1보다 큰 num_beams를 설정해 보세요.

output = model.generate(input_ids, # 주어진 텍스트
                        max_length=128, # 생성할 문장의 최대 길이
                        repetition_penalty=2.0, # 동일한 문구를 반복하여 생성할 때 부여할 벌점
                        num_beams=3
)

output_ids = output.numpy().tolist()[0]

# 생성한 문장 출력
tokenizer.decode(output_ids)

'근육이 커지기 위해서는 무엇보다 영양소가 풍부한 음식을 섭취하는 것이 중요하다. 면역력이 약한 어린이나 노약자 등은 균형 잡힌 식사를 통해 면역력을 높여야 한다.</d> 현대자동차가 지난달 국내 시장에서 전년 동월 대비 7.2% 감소한 5만5천750대를 판매했다고 1일 밝혔다.\n현대차는 지난달 국내 시장에서 전년 동월 대비 7.2% 감소한 4만5천800대를 판매했다.\n국내 시장에서는 전년 동월 대비 8.3% 감소한 6만3천990대를 판매했다.\n지난달 국내 시장에서 가장 많이 팔린 차종은 아반떼로 전년 동월 대비 6.9%'

### **Sampling**

앞의 두 기법은 실행할 때 마다 동일한 결과를 생성합니다. 반면 sampling은 확률에 따라 다음 단어를 랜덤하게 선택합니다.

![vanilla_sampling](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/sampling_search.png)



In [8]:
# 실행할 때마다 결과가 달라집니다.

output = model.generate(input_ids, # 주어진 텍스트
                        max_length=128, # 생성할 문장의 최대 길이
                        repetition_penalty=2.0, # 동일한 문구를 반복하여 생성할 때 부여할 벌점
                        do_sample=True # 이제 랜덤하게 단어가 선택됩니다.                        
)
 
output_ids = output.numpy().tolist()[0]

# 생성한 문장 출력
tokenizer.decode(output_ids)

'근육이 커지기 위해서는 몇 가지 노하우가 필요하다.\n바로 스트레스를 해소하는 것이다.\n먼저 지나친 운동은 좋지 못한 습관으로 이어질 수 있다.\n한 번에 많은 양의 영양이 빠져나가기 때문에 다이어트 중에도 충분한 효과를 기대하기 어려운 상황.\n또 스트레스는 우울해지는 세로토닌 분비를 증가시킨다.\n두 번째는 근육을 강화하는 방법이다.\n운동은 허리 주변 뼈의 탄력과 근육의 경직성을 줄여주고 척추에 혈액순환 능력을 향상시켜주며 디스크 탈출과 관절을 튼튼하게 해주는 효과가 있기 때문이다.\n마지막으로 체지방 감소를 위해서도 하루 2~3회 규칙적인 운동을 해줘야 한다.\n평소 걷도록 꾸준히 근력을 길러주는 것 또한 바람직하다는 것이 전문'